# <font color='navy'> Web Data Integration Project: Python code for data preprocessing and gold standards</font>

In [218]:
import numpy as np
import pandas as pd
import re

In [219]:
# Importing datasets
zomato_data = pd.read_csv("zomato.csv") 
yelp_data = pd.read_csv("yelp.csv")
yp_data = pd.read_csv("yellow_pages.csv")

# helper dataset
cities = pd.read_csv("uscities.csv")

# <font color='green'>Preprocessing for Schema Mapping stage</font>

### 1) Standardizing address for Zomato dataset

In [220]:
def tokenize(text): #tokenize
    result = [x.lower() for x in re.split(r'[,|;"]+', str(text))]
    return result

In [221]:
def extract_city(zomato_data):
    
    # create a list of cities that will serve as a dictionary
    cities_dict = cities["city"].to_numpy()
    cities_dict = [x.lower() for x in cities_dict]

    # tokenize address
    zomato_data['address_tok'] = zomato_data.apply(lambda x: tokenize(x['address']), axis=1)
    
    # check for city presence in address_tokens
    def if_city(tokens):
        last_element = tokens[-1].strip()
        if last_element in cities_dict:
            return last_element
    
    # extract city
    zomato_data['city'] = zomato_data.apply(lambda x: if_city(x['address_tok']), axis=1)
    
    # remove city from address
    def remove_city(data):
        for index, row in data.iterrows():
            tokens = row["address_tok"]
            if row["city"] is not None:
                del tokens[-1]
                
    remove_city(zomato_data)
    
    # join adress_tokens back to string and remove tokens from dataset
    zomato_data['address'] = zomato_data.apply(lambda x: ",".join(x['address_tok']), axis=1)
    del zomato_data["address_tok"]
    
    return zomato_data

In [222]:
zomato_data = extract_city(zomato_data)

### 2) Mapping missing cities in Zomato datasets using zip code

In [223]:
# get zip code and city dictionary

def getzipcitydict(citydata):
    # non_digit_zip_count counts the number of cities in cities dataset which does not have a numeric zipcode
    # for one row this is true
    non_digit_zip_count = 0
    dict_zip_city = {}
    for index1, row1 in citydata.iterrows(): 
        zips = str(row1['zips']).split(" ")
        for zip_each in zips:
            if zip_each.isdigit():
                dict_zip_city[int(zip_each)] = str(row1['city']).lower()
            else:
                non_digit_zip_count = non_digit_zip_count + 1
                #Attu Station has no zip code
                #print(row1['city'])           
    return dict_zip_city, non_digit_zip_count

In [224]:
dictzipcity, non_digit_zip_count = getzipcitydict(cities)
zomatocopy = zomato_data.copy()

In [225]:
def fillcitywithzipcodemapping(zipcode, city):
    try:
        #Fill the city data only if it is none. If not none it already contains a legal value and hence return the same 
        #legal value in the else case
        if city == None:
            zipcode =  int(zipcode)
            if zipcode in dictzipcity:
                return dictzipcity[zipcode]
        else:
            return city        
    except ValueError:
        return None 

In [226]:
zomatocopy['city'] = zomatocopy.apply(lambda x: fillcitywithzipcodemapping(x['zip'],x['city']), axis=1)
print("Rows in dataset with no city data after zip code mapping: " 
      + str(len(zomatocopy[zomatocopy.city.isnull()])))

Rows in dataset with no city data after zip code mapping: 0


### 3) Adding state attribute to Zomato dataset

In [227]:
state_dict = dict(zip(cities.city.str.lower(), cities.state_id))

def state(state_dict, city):
    if city in state_dict:
        return state_dict[city] 

zomatocopy['state'] = zomatocopy.apply(lambda x: state(state_dict, x['city']).lower(), axis=1)
zomato_data = zomatocopy

### 4) Temporary standardization of attrubutes' names to faciliate further operations on datasets 

In [229]:
# Yellow Pages
yp_data = yp_data.rename(columns = {'streetAddress':'address'})

# Yelp
yelp_data = yelp_data.rename(columns = {'telephone':'phone'})
yelp_data = yelp_data.rename(columns = {'streetAddress':'address'})

# Zomato
zomato_data = zomato_data.rename(columns = {'ID':'id'})

### 5) Extracting cuisine from 'category' attribute for Yellow Pages

We decided to pay more attention on preprocessing the cuisines, because they carry important information for the end user.

Zomato and Yelp have clean cuisine attributes, while in Yellow Pages the cuisine is hidden in the attribute 'category'. We want to extract cuisine from these categories. We noticed that typically cuisine is described as a noun (e.g. French, American, Korean) followed by a word 'restaurants'. Thus as a first step we want to extract the categories that contain word 'restaurant' (around 50). This is still not satisfactionary, because we are missig names such as 'pizza' or 'hamburgers' which also indicate type of food served. Knowing that Zomato and Yelp have it nicely organized, we create a cuisine dictionary consisted of unique cusine names for both Zomato and Yelp. Then, we check the remaining of YP categories against this dictionary. To sum up, the final attribute 'cuisine' for YP is composed of 1) names with 'restaurants' string 2) names that occur in Zomato and Yelp cuisines.

In [230]:
# Joint dictionary for cuisine yelp & zomato
def vocab(data, column):
    vocab = []
    tokens = data.apply(lambda x: tokenize(x[column]), axis=1)
    for i, j in tokens.iteritems():
        for token in j:
            if token not in vocab:
                vocab.append(token)        
    return vocab

def joint_vocab(vocab_1, vocab_2):
    cuisine_dict = list(vocab_1)
    cuisine_dict.extend(x for x in vocab_2 if x not in cuisine_dict)
    return sorted(cuisine_dict)
    
vocab_yelp = vocab(yelp_data, "category")
vocab_zomato = vocab(zomato_data, "cuisine")
vocab_yp = vocab(yp_data, "categories")

vocab = joint_vocab(vocab_yelp, vocab_zomato)

In [231]:
# Keep categories for Yellow Page that contain word "restaurant"
def trim_yp(vocab_yp):
    restaurants = []
    for i in range(len(vocab_yp)):
        if 'restaurant' in vocab_yp[i]:
            restaurants.append(vocab_yp[i])

    vocab_yp_wo_r = [x for x in vocab_yp if x not in restaurants]        
    vocab_yp_rest = [x for x in vocab_yp if x in vocab]
    vocab_yp = restaurants + vocab_yp_rest  

    return vocab_yp

In [232]:
def filter_categories(text):
    tokens = tokenize(text.lower())
    tokens = [x for x in tokens if x in vocab_yp]
    
    # removing "restaurants", blank spaces at the end of token and empty tokens (empty tokens are string that contain 
                                                                                 # only word 'restaurant')
    tokens = [x.replace('restaurants', '').strip() for x in tokens]
    
    while '' in tokens:
           tokens.remove('')
    return tokens

yp_data['cuisine'] = yp_data.apply(lambda x: filter_categories(x['categories']), axis=1)
yp_data['cuisine'] = yp_data.apply(lambda x: ",".join(x['cuisine']), axis=1)
del yp_data["categories"]

# <font color='green'>Preprocessing for Identity Resolution stage: Normalization</font>

In [233]:
from nltk.corpus import stopwords 

# Special characters - punctuation
def remove_special(text):
    text = str(text)
    return re.sub(r"[^a-zA-Z0-9]+", ' ', text)

# Abbreviation rules
def abb(text):
    replacements =  {"ave": "avenue", "st": "street", "blvd": "boulevard", 
                     "3rd": "third", "rd": "road", "ln": "lane", 
                     "hwy": "highway", "expy": "expressway", "dr": "drive", 
                     "pkwy": "parkway", "plz": "plaza", "pl": "place"}

    def replace(match):
        return replacements[match.group(0)]

    text = re.sub('|'.join(r'\b%s\b' % re.escape(s) for s in replacements), replace, text) 
    
    return text

# Stop words and space stripping
stop = set(stopwords.words('english'))  

def remove_stop_strip(t):
    t = t.split()
    filtered_words = []
    for word in t:
        if word not in stop:
            filtered_words.append(word.strip())
    return ' '.join(filtered_words)

In [234]:
datasets = [zomato_data, yp_data, yelp_data]
attriutes_to_normalize = ["name", "address", "city", "state"]

for i in datasets:
    for j in attriutes_to_normalize:
        j = str(j)
        i[j] = i.apply(lambda x: str((x[j])).lower(), axis=1)
        i[j] = i.apply(lambda x: remove_special(x[j]), axis=1)
        i[j] = i.apply(lambda x: abb(x[j]), axis=1)
        i[j] = i.apply(lambda x: remove_stop_strip(x[j]), axis=1)

In [239]:
# Previewing the data after preprocessing step

In [241]:
zomato_data.head(3)

,id,name,votes,rating,phone,address,zip,cuisine,reviewcount,city,state
0,0,strings ramen shop,15,3.1,(312) 374-3450,2141 archer avenue,60616,"Asian,Chinese,Ramen",2,chicago,il
1,1,francesco hole wall,179,4.0,(847) 272-0155,254 skokie boulevard,60062,Italian,6,northbrook,oh
2,3,four belly,12,3.0,(773) 661-6182,3227 n clark street,60657,"Asian,BBQ,Japanese",0,chicago,il


In [242]:
yp_data.head(3)

,id,name,address,city,state,zipCode,phone,website,priceRange,ratingValue,neighborhood,payment-method,years-in-business,extra-phones,aka,cuisine
0,1,full shilling,160 pearl street,new york,ny,10005,(212) 422-3855,http://www.thefullshilling.com,$$,4.0,Downtown Manhattan;Financial District,discover,16.0,Phone;(917) 962-0367;Phone;(212) 422-0036;Fax;...,The Full Shilling,"sandwich shops,take out,hamburgers & hot dogs,..."
1,2,dovetail,103 w 77th street,new york,ny,10024,(212) 362-3800,http://www.dovetailnyc.com;http://dovetailnyc....,$$$$,4.0,Upper Manhattan;Upper West Side,amex;master card;visa;diners club;discover;all...,8.0,NaN,NaN,"american,french,ice cream & frozen desserts,fi..."
2,3,patron mexican grill,608 9th avenue,new york,ny,10036,(212) 957-9050,http://www.patronnyc.com,$$,3.5,Hell's Kitchen;Midtown Manhattan,amex;visa;master card;all major credit cards,NaN,Phone;(917) 791-5098;Fax;(212) 957-4047,NaN,"mexican,latin american,bar & grills,take out"


In [243]:
yelp_data.head(3)

,id,name,address,city,state,zipCode,phone,website,priceRange,category,...,Accepts Credit Cards,Good For,Parking,Attire,Ambience,Alcohol,Outdoor Seating,Wi-Fi,Waiter Service,Caters
0,1,sunshine co,780 washington avenue,new york,ny,11238.0,(347) 750-5275,sunshinecobk.com,$$,American (New);Cocktail Bars,...,Yes,Brunch,Street,Casual,Hipster,Full Bar,Yes,Free,Yes,No
1,2,adella,410 w 43rd street,new york,ny,10036.0,(212) 273-0737,adellanyc.com,$$,Tapas Bars;American (New);Wine Bars,...,Yes,Dessert,Street,Casual,Trendy,Beer & Wine Only,Yes,No,Yes,No
2,3,rex,864 10th avenue,new york,ny,10019.0,(929) 900-5784,rexcoffeenyc.com,$,Coffee & Tea;Sandwiches,...,Yes,Breakfast,Street,Casual,Hipster,No,No,Free,No,Yes


# <font color='green'>Gold standards</font>

In [210]:
# Zomato and Yellow Pages

    # 1) matching record pairs (randomly chosen, 20%):
        # a) exact matches on our matching criteria: name, address, city, state (exact entities)
        # b) matches on phone with HIGH Jaacard coefficient for the matching criteria (almost exact entities)
        
    # 2) corner cases (30%):
        # a) matches on phone with MEDIUM Jaacard coefficient for the mathich criteria  
        
    # 3) non-matching record pairs (randomly chosen, 50% ):
        # a) randomly generated pairs of entities for two data-set, labelled manually (different entities)
        # b) entities with same attribute name (e.g. same restaurant, same address), but LOW Jaacard coefficient 
            # for another attributes (different entities with small similarity)
        

In [245]:
# Jaacard similarity
def get_jaccard_sim(str1, str2):
    str1 = str(str1)
    str2 = str(str2)
    a = set(str1.split()) 
    b = set(str2.split())
    c = a.intersection(b)
    
    if (len(a) + len(b) - len(c)) == 0:
        return 0
    else:
        return float(len(c)) / (len(a) + len(b) - len(c))

In [246]:
# Get records not matched on attribute
def false_negatives(df1_, df2_):

    # change ids
    df1 = df1_.copy()
    df2 = df2_.copy()
    
    merge = pd.merge(df1,df2,on='phone') 
    
    # check if attribute name is the same
    def same_name(text1, text2):
        text1 = str(text1)
        text2 = str(text2)
        return 1 if text1 != text2 else 0
    
    merge["if_same_name"] = merge.apply(lambda x: same_name(x["name_x"], x["name_y"]), axis=1)
    merge["if_same_address"] = merge.apply(lambda x: same_name(x["address_x"], x["address_y"]), axis=1)
    merge_ = merge[(merge["if_same_address"] == 1) | (merge["if_same_address"] == 1)]
    
    # compute Jaccard distance
    merge["jaccard_name"] = merge.apply(lambda x: round(get_jaccard_sim(x["name_x"], x["name_y"]),1), axis=1)                                         
    merge["jaccard_address"] = merge.apply(lambda x: round(get_jaccard_sim(x["address_x"], x["address_y"]),1), axis=1)
    merge["jaccard_sum"] = merge.apply(lambda x: round(int(x["jaccard_name"]) + int(x["jaccard_address"])), axis=1)
    merge["jaccard_tot"] = merge.apply(lambda x: round((int(x["jaccard_name"]) + int(x["jaccard_address"]))/2,2), axis=1)
    
    merge = merge[["id_x", "id_y", "name_x", "name_y", "address_x", "address_y", "jaccard_name", "jaccard_address", "jaccard_tot",
                  "jaccard_sum"]]
    merge.sort_values(by=['jaccard_tot'], inplace=True,ascending=False)
    
    return merge

In [247]:
merge = false_negatives(zomato_data, yp_data)
merge.to_csv('zomato_yellow_pages_gs.csv')
merge

,id_x,id_y,name_x,name_y,address_x,address_y,jaccard_name,jaccard_address,jaccard_tot,jaccard_sum
2463,7689,811,buddakan,buddakan,75 9th avenue,75 9th avenue,1.0,1.0,1.0,2
1842,4208,3007,chalet edelweiss,chalet edelweiss,8740 sepulveda boulevard,8740 sepulveda boulevard,1.0,1.0,1.0,2
1835,4191,3802,tatsu ramen,tatsu ramen,7111 melrose avenue,7111 melrose avenue,1.0,1.0,1.0,2
1836,4195,3184,hart hunter,hart hunter,7950 melrose avenue,7950 melrose avenue,1.0,1.0,1.0,2
861,1872,26042,greenville avenue pizza company,greenville avenue pizza company,1923 greenville avenue,1923 greenville avenue,1.0,1.0,1.0,2
...,...,...,...,...,...,...,...,...,...,...
746,1812,15454,n,n burger,7909 lbj freeway,12413 n tatum boulevard,0.5,0.0,0.0,0
745,1812,15059,n,n burger,7909 lbj freeway,9585 w camelback road,0.5,0.0,0.0,0
743,1743,27455,n,n burger,7940 n central expressway,550 newhall drive,0.5,0.0,0.0,0
742,1743,27043,n,n burger,7940 n central expressway,2950 e capitol expressway,0.5,0.1,0.0,0


In [250]:
# Now get randomly generate pairs of dataset
def random_pairs(df1_, df2_):
    # change ids
    df1 = df1_.copy()
    df2 = df2_.copy()
    df2["id"] = df2.apply(lambda x: "yellow_pages_" + str(x["id"]), axis=1)
    df1["id"] = df1.apply(lambda x: "zomato_" + str(x["id"]), axis=1)
    
    df1 = df1[["id", "name", "address"]]
    df2 = df2[["id", "name", "address"]]
    
    horizontal_stack = pd.concat([df1, df2], axis=1)
    
    return horizontal_stack

In [251]:
concat = random_pairs(zomato_data, yp_data)
concat.to_csv('zomato_yellow_pages_gs_random.csv')

In [2]:
def false_positives(df1, df2, join_attribute, distinguishable_attribute):
    
    merge = pd.merge(df1,df2,on=join_attribute)
    
    def same_name(text1, text2):
        return 1 if text1 != text2 else 0
    
    merge["if_same_" + distinguishable_attribute] = merge.apply(lambda x: same_name(x[distinguishable_attribute + "_x"], 
                                                                              x[distinguishable_attribute + "_y"]), axis=1)  
    
    merge_ = merge[merge["if_same_" + distinguishable_attribute] == 1]    
    
    # compute Jaccard distance
    merge_["jaccard_" + distinguishable_attribute] = merge.apply(lambda x: round(get_jaccard_sim(x[distinguishable_attribute + "_x"], 
                                                                              x[distinguishable_attribute + "_y"]),1), axis=1)
    merge_ = merge_[[join_attribute,
                     distinguishable_attribute + "_x", 
                     distinguishable_attribute + "_y", 
                     "jaccard_" + distinguishable_attribute]]
    
    return merge_ 

In [46]:
# Now we can choose high Jaccard coefficient to search for strong false positives
fp = false_positives(zomato_data, yp_data, "name", "address")
fp = fp[(fp["jaccard_address"] > 0) & (fp["jaccard_address"] <=0.2)]
fp.sort_values(by=['jaccard_address'], inplace=True,ascending=False)
fp.head(50)

# for example "qdoba mexican grill" has addresses "655 s hope st" and "230 s 40th st" with Jaccard coefficient 0.3

# another example "el compadre" has addresses "7408 sunset blvd" and "1449 w sunset blvd" with Jaccard coefficient 0.4

C:\Users\48668\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,name,address_x,address_y,jaccard_address
4504,in-n-out burger,7009 sunset blvd,21001 n tatum blvd,0.2
3807,maria's italian kitchen,13353 ventura blvd,10761 w pico blvd,0.2
3768,planet dailies,"6333 w. 3rd st., ste. o20",6333 w 3rd st ste 175l,0.2
3751,tomato pie pizza joint,7751 melrose ave,2457 hyperion ave,0.2
5636,papa john's pizza,6619 mckee rd,7708 regents rd,0.2
5637,papa john's pizza,6619 mckee rd,8665 navajo rd,0.2
5640,papa john's pizza,6619 mckee rd,14092 preston rd,0.2
5643,papa john's pizza,6619 mckee rd,4417 s lancaster rd,0.2
5660,papa john's pizza,6615 university ave,3621 aramingo ave,0.2
5667,papa john's pizza,6615 university ave,1810 w northern ave,0.2
